In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

tf.test.is_gpu_available()

/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhenwuzhou/environment/tf1_py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zhenwuzhou/environment/tf1_py3

1.13.1
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


False

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test, y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)
print(np.max(x_train),np.min(x_train))

# 这里对label数据类型进行更改，
y_train = np.asarray(y_train,dtype = np.int64)
y_valid = np.asarray(y_valid,dtype = np.int64)
y_test = np.asarray(y_test,dtype = np.int64)

(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)
255 0


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train:[None,28,28] -> [None,784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_valid_scaled = scaler.fit_transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_test_scaled = scaler.fit_transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [4]:
def make_dataset(images,labels,epochs,batch_size,shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    if shuffle:
        dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [5]:
# 我们并不能像在2.0中那样通过eager模式来访问dataset
epochs = 10
batch_size = 20
dataset = make_dataset(x_train_scaled,y_train,
                       epochs = epochs,
                       batch_size = batch_size)
for data, label in dataset.take(1):
    print(data)
    print(label)

RuntimeError: dataset.__iter__() is only supported when eager execution is enabled.

In [10]:
# 那么要怎样才能在1.0中访问dataset呢
epochs = 10
batch_size = 20
dataset = make_dataset(x_train_scaled,y_train,
                       epochs = epochs,
                       batch_size = batch_size)
# 首先需要创建dataset_iter
# make_one_shot_iterator有下面这些特点:
# 1、自动初始化
# 2、不能被重新初始化；(make_initializable_iterator是可以重新被初始化的)
dataset_iter = dataset.make_one_shot_iterator()
x,y = dataset_iter.get_next() # 取出tensor
# 创建session来访问
with tf.Session() as sess:
    x_val,y_val = sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)

(20, 784)
(20,)


In [11]:
# 构建模型
hidden_units = [100,100]
class_num = 10 # 10分类

input_for_next_layer = x
# 隐藏层
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer,
                                           hidden_unit,
                                           activation=tf.nn.relu)
# 输出层
logits = tf.layers.dense(input_for_next_layer,
                         class_num)

# loss计算
# 1. 将最后一层的输出做softmax计算出prob(每种分类的概率)
# 2. labels -> one_hot 将标签进行one-hot编码
# 3. 计算loss：就是去概率最大的那个max_prob进行-log(max_prob)的计算
loss = tf.losses.sparse_softmax_cross_entropy(labels = y,
                                              logits = logits)

# 计算正确率accuracy
# 取出概率最大的那个位置的索引位置作为预测
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction,y) # 正确的为1，错误的为0
# 先将int值转为float类型然后在进行正确率的计算
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64)) 

# 定义train_op,在训练的时候就是调用train_op
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [12]:
print(x)
print(logits)

Tensor("IteratorGetNext_1:0", shape=(?, 784), dtype=float32)
Tensor("dense_5/BiasAdd:0", shape=(?, 10), dtype=float32)


In [13]:
# 模型训练
init = tf.global_variables_initializer()
train_steps_per_epoch = x_train_scaled.shape[0] // batch_size

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            loss_val,accuracy_val,_ = sess.run(
                [loss,accuracy,train_op])
            print("\r[Train] epoch: %d, step: %d, loss: %3.5f, accuracy: %2.2f" % 
                  (epoch,step,loss_val,accuracy_val),end=" ")

[Train] epoch: 0, step: 315, loss: 0.61710, accuracy: 0.75 

[Train] epoch: 0, step: 636, loss: 0.59854, accuracy: 0.80 

[Train] epoch: 0, step: 951, loss: 0.55254, accuracy: 0.90 

[Train] epoch: 0, step: 1260, loss: 0.43066, accuracy: 0.85 

[Train] epoch: 0, step: 1581, loss: 0.38841, accuracy: 0.85 

...

[Train] epoch: 9, step: 1764, loss: 0.20248, accuracy: 0.95 

[Train] epoch: 9, step: 2075, loss: 0.04230, accuracy: 1.00 

[Train] epoch: 9, step: 2380, loss: 0.30686, accuracy: 0.90 

[Train] epoch: 9, step: 2676, loss: 0.20350, accuracy: 0.90 

[Train] epoch: 9, step: 2749, loss: 0.08716, accuracy: 0.90 